# MNIST by Convolution NN (LeNet-5)
* 2 conv([5,5],64) 2 maxpooling(strides=2) FC(1024)
* input=[batch,length,width,deep]
* Convolution kernel(w) = [length,width,deep_in,deep_out]
* strides = [1,step,step,1]
* new_height = new_width = (input — kernel + 1) / Strides

In [1]:
#import data and library
import numpy as np
import pandas as pd 
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
test_data = mnist.test.images  # Returns np.array
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)

tf.logging.set_verbosity(old_v)

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#def feature map and pooling


def weight_variable(name,shape):
    initializer = tf.contrib.layers.xavier_initializer(seed=1)
    return tf.get_variable(name,shape,initializer=initializer,dtype=tf.float32)

def bias_variable(name,shape):
    return tf.get_variable(name, shape)

def conv2d(inputs, w):
    return tf.nn.conv2d(inputs,w,strides = [1,1,1,1],padding = 'SAME')

def max_pooling_2x2(inputs,x=2,y=2):
    ksize = [1,x,y,1]
    return tf.nn.max_pool(inputs,ksize,strides = [1,2,2,1],padding = 'SAME')

In [32]:
#set outlines 
inputs_num = 784 # 28*28 =784  (784,1)
outputs_num = 10 # 0 to 9   

#reset variable in tensorflow
tf.reset_default_graph()

#stack data
X = np.vstack([mnist.train.images, mnist.test.images, mnist.validation.images])
y = np.vstack([mnist.train.labels, mnist.test.labels, mnist.validation.labels])

#set placeholder(input and target)
inputs= tf.placeholder(tf.float32,[None,inputs_num],)
targets= tf.placeholder(tf.float32,[None,outputs_num],)
#transfer image to 4D data([batch,in_height,in_width,in_channels])
inputs_image = tf.reshape(inputs,[-1,28,28,1])

#set 1st convolution layer w and b
w_conv1 = weight_variable('w_1',[5,5,1,32])
b_conv1 = bias_variable('b_1',[32])
#set 2nd convolution layer w and b
w_conv2 = weight_variable('w_2',[5,5,32,64])
b_conv2 = bias_variable('b_2',[64])

#convolution layer and max pooling (relu activation function) 2 times pooling 28 ->14 ->7
h_conv1 = tf.nn.relu(conv2d(inputs_image,w_conv1)+b_conv1)
h_pool1 = max_pooling_2x2(h_conv1)

h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
h_pool2 = max_pooling_2x2(h_conv2)

#set 1st fully connected variable(last layer 7*7*64, 1024 nuron)
fc_w1 = weight_variable('fc_w1',[7*7*64,1024])
fc_b1 = bias_variable('fc_b1',[1024])

#flatten
h_pool2_flatten = tf.reshape(h_pool2,[-1,7*7*64])

# fully connected 1
fc_1 = tf.nn.relu(tf.matmul(h_pool2_flatten,fc_w1)+fc_b1)

# dropout(testing need turn off)
keep_prop = tf.placeholder(tf.float32)
fc_1_drop = tf.nn.dropout(fc_1,keep_prop)

#set 2nd fully connected variable(last layer 1024, 10 nuron)
fc_w2 = weight_variable('fc_w2',[1024,10])
fc_b2 = bias_variable('fc_b2',[10])

#predicted outputs
outputs = tf.nn.softmax(tf.matmul(fc_1_drop,fc_w2)+fc_b2)

# set loss and optimize object
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=targets))
# loss = tf.reduce_mean(-tf.reduce_sum(targets * tf.log(outputs), 1))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001,).minimize(loss)

#after softmax, let probability to 0 or 1 by argmax function, and judge between target and output
out_eql_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

#predicte accuracy (need cast to float32)
accuracy = tf.reduce_mean(tf.cast(out_eql_target,tf.float32))

#框架完成

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# session = tf.Session(config=config)
#sess
sess= tf.InteractiveSession()

#initial variable
initializer = tf.global_variables_initializer()
sess.run(initializer)


#set training paramerter
epochs = 10000
batch_num = 256
batch_cycle_a = mnist.train._num_examples // batch_num
batch_cycle_b = mnist.validation._num_examples // batch_num
batch_cycle_c = mnist.test._num_examples // batch_num
prev_validation = 9999999.

#Training model
import time
start = time.time()
for epoch_counter in range(epochs):
    curr_epoc_loss = 0.
    curr_epoc_val = 0.
    curr_epoc_tes = 0.
    for batch_counter in range(batch_cycle_a):
        input_batch, target_batch = mnist.train.next_batch(batch_num)
        _,batch_loss,train_accuracy = sess.run([optimizer,loss,accuracy],
                                feed_dict={inputs:input_batch,targets: target_batch,keep_prop:0.5})
        curr_epoc_loss += batch_loss
    curr_epoc_loss/=batch_cycle_a
    # validation
    for batch_counter in range(batch_cycle_b):
        input_batch, target_batch = mnist.validation.next_batch(batch_num)
        val_loss, val_accuracy = sess.run([loss,accuracy],feed_dict={inputs:input_batch,targets:target_batch, keep_prop:0.5})
        curr_epoc_val += val_loss
    curr_epoc_val/=batch_cycle_b
#     if val_loss>prev_validation:
#         break
#     prev_validation = val_loss


    print(f'Epoch cycle: {epoch_counter + 1}'+
             f' Training loss: {curr_epoc_loss:.3f}'+
             f' Validation loss: {val_loss:.3f}'+
             f' Validation accuracy: {val_accuracy*100.:.2f}'+'%'+
             f' times: {time.time() - start}')
    
    
    for batch_counter in range(batch_cycle_c):
        input_batch, target_batch = mnist.test.next_batch(batch_num)
        test_accuracy = sess.run([accuracy],feed_dict ={inputs:input_batch,targets:target_batch,keep_prop:1})
    test_accuracy_percent=test_accuracy[0]*100.
    print(f'test accuracy: {test_accuracy_percent:.2f}')
    if  test_accuracy[0] > 0.997 and val_accuracy > 0.997:
        saver = tf.train.Saver()
        save_path = saver.save(sess, 'mnist_cnn_model/mnist_cnn.ckpt')
        print('Save to path:', save_path)
        break
print('end of training')





E:\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch cycle: 1 Training loss: 1.857 Validation loss: 1.597 Validation accuracy: 88.28% times: 6.499316692352295
test accuracy: 90.23
Epoch cycle: 2 Training loss: 1.565 Validation loss: 1.517 Validation accuracy: 96.09% times: 13.414217472076416
test accuracy: 93.75
Epoch cycle: 3 Training loss: 1.531 Validation loss: 1.513 Validation accuracy: 96.48% times: 20.34702777862549
test accuracy: 95.31
Epoch cycle: 4 Training loss: 1.512 Validation loss: 1.492 Validation accuracy: 97.66% times: 27.212347984313965
test accuracy: 99.61
Epoch cycle: 5 Training loss: 1.502 Validation loss: 1.498 Validation accuracy: 96.88% times: 34.066669940948486
test accuracy: 96.09
Epoch cycle: 6 Training loss: 1.495 Validation loss: 1.492 Validation accuracy: 96.88% times: 40.94104194641113
test accuracy: 98.44
Epoch cycle: 7 Training loss: 1.492 Validation loss: 1.488 Validation accuracy: 97.66% times: 47.81301951408386
test accuracy: 98.44
Epoch cycle: 8 Training loss: 1.488 Validation loss: 1.489 Validat

Epoch cycle: 62 Training loss: 1.464 Validation loss: 1.467 Validation accuracy: 99.61% times: 434.043092250824
test accuracy: 99.22
Epoch cycle: 63 Training loss: 1.464 Validation loss: 1.465 Validation accuracy: 99.61% times: 441.0762026309967
test accuracy: 99.61
Epoch cycle: 64 Training loss: 1.464 Validation loss: 1.474 Validation accuracy: 98.83% times: 448.1054677963257
test accuracy: 98.05
Epoch cycle: 65 Training loss: 1.464 Validation loss: 1.468 Validation accuracy: 99.22% times: 455.16351318359375
test accuracy: 100.00
Epoch cycle: 66 Training loss: 1.464 Validation loss: 1.474 Validation accuracy: 98.44% times: 462.24549174308777
test accuracy: 98.83
Epoch cycle: 67 Training loss: 1.464 Validation loss: 1.476 Validation accuracy: 98.44% times: 469.28014183044434
test accuracy: 99.22
Epoch cycle: 68 Training loss: 1.464 Validation loss: 1.472 Validation accuracy: 98.83% times: 476.3112874031067
test accuracy: 99.61
Epoch cycle: 69 Training loss: 1.464 Validation loss: 1.471

Epoch cycle: 123 Training loss: 1.463 Validation loss: 1.473 Validation accuracy: 98.83% times: 864.0166194438934
test accuracy: 99.61
Epoch cycle: 124 Training loss: 1.463 Validation loss: 1.467 Validation accuracy: 99.61% times: 871.075658082962
test accuracy: 98.83
Epoch cycle: 125 Training loss: 1.463 Validation loss: 1.466 Validation accuracy: 100.00% times: 878.1227300167084
test accuracy: 100.00
Save to path: mnist_cnn_model/mnist_cnn.ckpt
end of training


In [33]:
# save prediction

#---------------------------------------------------------------#
import pandas as pd

test_df = pd.read_csv('test.csv')
X = test_df.values
print('特征矩阵的形状：', X.shape)
X1 = X[:1000]
X2 = X[1000:2000]
X3 = X[2000:3000]
X4 = X[3000:4000]
X5 = X[4000:5000]
X6 = X[5000:6000]
X7 = X[6000:7000]
X8 = X[7000:8000]
X9 = X[8000:9000]
X10 = X[9000:10000]
X11 = X[10000:11000]
X12 = X[11000:12000]
X13 = X[12000:13000]
X14 = X[13000:14000]
X15 = X[14000:15000]
X16 = X[15000:16000]
X17 = X[16000:17000]
X18 = X[17000:18000]
X19 = X[18000:19000]
X20 = X[19000:20000]
X21 = X[20000:21000]
X22 = X[21000:22000]
X23 = X[22000:23000]
X24 = X[23000:24000]
X25 = X[24000:25000]
X26 = X[25000:26000]
X27 = X[26000:27000]
X28 = X[27000:]

y1 = sess.run(outputs, feed_dict={inputs:X1,keep_prop:1})
y2 = sess.run(outputs, feed_dict={inputs:X2,keep_prop:1})
y3 = sess.run(outputs, feed_dict={inputs:X3,keep_prop:1})
y4 = sess.run(outputs, feed_dict={inputs:X4,keep_prop:1})
y5 = sess.run(outputs, feed_dict={inputs:X5,keep_prop:1})
y6 = sess.run(outputs, feed_dict={inputs:X6,keep_prop:1})
y7 = sess.run(outputs, feed_dict={inputs:X7,keep_prop:1})
y8 = sess.run(outputs, feed_dict={inputs:X8,keep_prop:1})
y9 = sess.run(outputs, feed_dict={inputs:X9,keep_prop:1})
y10 = sess.run(outputs, feed_dict={inputs:X10,keep_prop:1})
y11 = sess.run(outputs, feed_dict={inputs:X11,keep_prop:1})
y12 = sess.run(outputs, feed_dict={inputs:X12,keep_prop:1})
y13 = sess.run(outputs, feed_dict={inputs:X13,keep_prop:1})
y14 = sess.run(outputs, feed_dict={inputs:X14,keep_prop:1})
y15 = sess.run(outputs, feed_dict={inputs:X15,keep_prop:1})
y16 = sess.run(outputs, feed_dict={inputs:X16,keep_prop:1})
y17 = sess.run(outputs, feed_dict={inputs:X17,keep_prop:1})
y18 = sess.run(outputs, feed_dict={inputs:X18,keep_prop:1})
y19 = sess.run(outputs, feed_dict={inputs:X19,keep_prop:1})
y20 = sess.run(outputs, feed_dict={inputs:X20,keep_prop:1})
y21 = sess.run(outputs, feed_dict={inputs:X21,keep_prop:1})
y22 = sess.run(outputs, feed_dict={inputs:X22,keep_prop:1})
y23 = sess.run(outputs, feed_dict={inputs:X23,keep_prop:1})
y24 = sess.run(outputs, feed_dict={inputs:X24,keep_prop:1})
y25 = sess.run(outputs, feed_dict={inputs:X25,keep_prop:1})
y26 = sess.run(outputs, feed_dict={inputs:X26,keep_prop:1})
y27 = sess.run(outputs, feed_dict={inputs:X27,keep_prop:1})
y28 = sess.run(outputs, feed_dict={inputs:X28,keep_prop:1})
import numpy as np
y = np.vstack([y1, y2, y3, y4, y5, y6, y7, y8, y9, y10, y11, y12, y13, y14, y15, y16, y17, y18, y19, y20, y21, y22, y23, y24, y25, y26, y27, y28])
y_argmax = np.argmax(y, 1)
y_argmax.shape
print('predicted shape：', y_argmax.shape)
commit_df = pd.DataFrame({'ImageId': range(1, 1+len(y_argmax)),
                         'Label': y_argmax})
fileName = 'kaggle_commit.csv'
commit_df.to_csv(fileName, index=False)
print('predicted 结果已经保存到文件', fileName)


特征矩阵的形状： (28000, 784)
predicted shape： (28000,)
predicted 结果已经保存到文件 kaggle_commit.csv


In [ ]:
#---------------------------------------------施工中-------------------------------------------------------------#
#CNN model

import numpy as np
import tensorflow as tf

def weight_variable(shape):
    initializer = tf.contrib.layers.xavier_initializer()
    return tf.get_variable('weight',shape,initializer=initializer,dtype=tf.float32)

def bias_variable(shape):
    initialier = tf.constance_initializer()
    return tf.get_variable('bias',shape,initializer=initializer,dtype=tf.float32)

def conv2d(inputs, w):
    return tf.nn.conv2d(inputs,w,strides = [1,1,1,1],padding = 'SAME')

def max_pooling_2x2(inputs,x=2,y=2):
    ksize = [1,x,y,1]
    return tf.nn.max_pool(inputs,ksize,strides = [1,2,2,1],padding = 'SAME')

class MNISTcnn(object):
    def __init__(self,inputs,targets,conf):
        self.inputs = inputs
        self.targets = targets
        self.keep_drop = tf.palceholder(tf.float32)
        
        # 1st layer
        with tf.variable_scope(conv1):
            W_conv1 = weight_variable([5, 5, 1, 32])
            b_conv1 = bias_variable([32])
            h_conv1 = tf.nn.relu(conv2d(self.inputs, W_conv1) + b_conv1)
            h_pool1 = max_pool_2x2(h_conv1)
        # 2nd layer
        with tf.variable__scope(conv2):
            W_conv2 = weight_variable([5, 5, 32, 64])
            b_conv2 = bias_variable([64])
            h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
            h_pool2 = max_pool_2x2(h_conv2)
        #fully connected 1st layer
        with tf.variable_scope("fc1"):
            shape = int(np.prod(h_pool2.get_shape()[1:]))
            W_fc1 = weight_variable([shape, 1024])
            b_fc1 = bias_variable([1024])
            h_pool2_flat = tf.reshape(h_pool2, [-1, shape])
            h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        # dropout
        h_fc1_drop = tf.nn.dropout(h_fc1, self.keep_prob)
        #fully connected to outputs
        with tf.variable_scope("fc2"):
            W_fc2 = weight_variable([1024, 10])
            b_fc2 = bias_variable([10])
            outputs = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.targets, logits=outputs))
        self.pred = tf.argmax(outputs, 1)

        self.correct_prediction = tf.equal(tf.argmax(outputs,1), tf.argmax(self.targets,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))